# Step 3: Model Building

Using the labeled feature data set constructed in the `Code/2_feature_engineering.ipynb` Jupyter notebook, this notebook loads the data from the Azure Blob container and splits it into a training and test data set. We then build a machine learning model (a decision tree classifier or a random forest classifier) to predict when different components within our machine population will fail. We store the better performing model for deployment in an Azure web service in the next. We will prepare and build the web service in the `Code/4_operationalization.ipynb` Jupyter notebook.

**Note:** This notebook will take about 2-4 minutes to execute all cells, depending on the compute configuration you have setup.

In [2]:
# import the libraries
import time

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
# for creating pipelines and model
from pyspark.ml.feature import StringIndexer, VectorAssembler, VectorIndexer

# For some data handling

# Time the notebook execution. 
# This will only make sense if you "Run all cells"
tic = time.time()

# This is the final feature data file.
TRAINING_TABLE = 'training_data'
TESTING_TABLE = 'testing_data'
model_type = 'RandomForest' # Use 'DecisionTree' or 'GBTClassifier' or 'RandomForest'

In [3]:
dbutils.widgets.removeAll()
dbutils.widgets.text("TRAINING_TABLE",TRAINING_TABLE)
dbutils.widgets.text("model", model_type)

Load the data and dump a short summary of the resulting DataFrame.

In [5]:
train_data = spark.sql("SELECT * FROM " + dbutils.widgets.get("TRAINING_TABLE"))
train_data.limit(10).toPandas().head(10)

Out[ 3 ]: 
 machineID dt_truncated volt_rollingmean_12 rotate_rollingmean_12 \
0 45 2015-10-30 00:00:00 174.611847 450.720590 
1 45 2015-10-29 12:00:00 166.425099 441.652415 
2 45 2015-10-29 00:00:00 169.575066 435.441998 
3 45 2015-10-28 12:00:00 169.190066 448.272390 
4 45 2015-10-28 00:00:00 172.485177 466.131104 
5 45 2015-10-27 12:00:00 167.217351 442.999995 
6 45 2015-10-27 00:00:00 167.309299 432.430837 
7 45 2015-10-26 12:00:00 172.232218 445.012891 
8 45 2015-10-26 00:00:00 168.922431 456.224824 
9 45 2015-10-25 12:00:00 173.080955 455.142128 

 pressure_rollingmean_12 vibration_rollingmean_12 volt_rollingmean_24 \
0 102.291328 40.325912 169.913411 
1 99.466725 39.203922 168.000082 
2 99.452056 41.994982 169.382566 
3 101.831761 40.411471 170.837621 
4 103.078662 41.311734 169.851264 
5 97.828752 39.624587 167.263325 
6 102.413577 40.168805 169.770759 
7 96.921228 39.730115 170.577325 
8 99.373114 39.491180 171.001693 
9 99.371852 39.216454 173.736938 

 rotate_rollingmean_24 pressure_rollingmean_24 vibration_rollingmean_24 \
0 447.822531 100.186140 39.411553 
1 438.547206 99.459390 40.599452 
2 441.857194 100.641908 41.203226 
3 457.201747 102.455211 40.861602 
4 454.565550 100.453707 40.468160 
5 437.715416 100.121164 39.896696 
6 438.721864 99.667402 39.949460 
7 450.618858 98.147171 39.610648 
8 455.683476 99.372483 39.353817 
9 454.410161 105.057001 39.066713 

 ... error5sum_rollingmean_24 comp1sum comp2sum comp3sum comp4sum \
0 ... 0.0 411.0 396.0 321.0 516.0 
1 ... 0.0 410.0 395.0 320.0 515.0 
2 ... 0.0 410.0 395.0 320.0 515.0 
3 ... 0.0 409.0 394.0 319.0 514.0 
4 ... 0.0 409.0 394.0 319.0 514.0 
5 ... 0.0 408.0 393.0 318.0 513.0 
6 ... 0.0 408.0 393.0 318.0 513.0 
7 ... 0.0 407.0 392.0 317.0 512.0 
8 ... 0.0 407.0 392.0 317.0 512.0 
9 ... 0.0 406.0 391.0 316.0 511.0 

 model age model_encoded failure label_e 
0 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
1 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
2 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
3 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
4 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
5 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
6 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
7 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
8 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 
9 model1 14 (0.0, 0.0, 0.0) 0.0 0.0 

[10 rows x 40 columns]

# Prepare the Training/Testing data

A fundamental practice in machine learning is to calibrate and test your model parameters on data that has not been used to train the model. Evaluation of the model requires splitting the available data into a training portion, a calibration portion and an evaluation portion. Typically, 80% of data is used to train the model and 10% each to calibrate any parameter selection and evaluate your model.

In general random splitting can be used, but since time series data have an inherent correlation between observations. For predictive maintenance problems, a time-dependent spliting strategy is often a better approach to estimate performance. For a time-dependent split, a single point in time is chosen, the model is trained on examples up to that point in time, and validated on the examples after that point. This simulates training on current data and score data collected in the future data after the splitting point is not known. However, care must be taken on labels near the split point. In this case, feature records within 7 days of the split point can not be labeled as a failure, since that is unobserved data. 

In the following code blocks, we split the data at a single point to train and evaluate this model.

In [8]:
# define list of input columns for downstream modeling

# We'll use the known label, and key variables.
label_var = ['label_e']
key_cols =['machineID','dt_truncated']

# Then get the remaing feature names from the data
input_features = train_data.columns

# We'll use the known label, key variables and 
# a few extra columns we won't need.
remove_names = label_var + key_cols + ['failure','model_encoded','model' ]

# Remove the extra names if that are in the input_features list
input_features = [x for x in input_features if x not in set(remove_names)]

input_features

Out[ 4 ]: 
['volt_rollingmean_12',
 'rotate_rollingmean_12',
 'pressure_rollingmean_12',
 'vibration_rollingmean_12',
 'volt_rollingmean_24',
 'rotate_rollingmean_24',
 'pressure_rollingmean_24',
 'vibration_rollingmean_24',
 'volt_rollingmean_36',
 'vibration_rollingmean_36',
 'rotate_rollingmean_36',
 'pressure_rollingmean_36',
 'volt_rollingstd_12',
 'rotate_rollingstd_12',
 'pressure_rollingstd_12',
 'vibration_rollingstd_12',
 'volt_rollingstd_24',
 'rotate_rollingstd_24',
 'pressure_rollingstd_24',
 'vibration_rollingstd_24',
 'volt_rollingstd_36',
 'rotate_rollingstd_36',
 'pressure_rollingstd_36',
 'vibration_rollingstd_36',
 'error1sum_rollingmean_24',
 'error2sum_rollingmean_24',
 'error3sum_rollingmean_24',
 'error4sum_rollingmean_24',
 'error5sum_rollingmean_24',
 'comp1sum',
 'comp2sum',
 'comp3sum',
 'comp4sum',
 'age']

Spark models require a vectorized data frame. We transform the dataset here and then split the data into a training and test set. We use this split data to train the model on 9 months of data (training data), and evaluate on the remaining 3 months (test data) going forward.

In [10]:
# assemble features
va = VectorAssembler(inputCols=(input_features), outputCol='features')
train_data = va.transform(train_data).select('machineID','dt_truncated','label_e','features')

# set maxCategories so features with > 10 distinct values are treated as continuous.
featureIndexer = VectorIndexer(inputCol="features", 
                               outputCol="indexedFeatures", 
                               maxCategories=10).fit(train_data)

# fit on whole dataset to include all labels in index
labelIndexer = StringIndexer(inputCol="label_e", outputCol="indexedLabel").fit(train_data)

training = train_data

print(training.count())

604307

# Classification models

A particualar troubling behavior in predictive maintenance is machine failures are usually rare occurrences compared to normal operation. This is fortunate for the business as maintenance and saftey issues are few, but causes an imbalance in the label distribution. This imbalance leads to poor performance as algorithms tend to classify majority class examples at the expense of minority class, since the total misclassification error is much improved when majority class is labeled correctly. This causes low recall or precision rates, although accuracy can be high. It becomes a larger problem when the cost of false alarms is very high. To help with this problem, sampling techniques such as oversampling of the minority examples can be used. These methods are not covered in this notebook. Because of this, it is also important to look at evaluation metrics other than accuracy alone.

We will build and compare two different classification model approaches:

 - **Decision Tree Classifier**: Decision trees and their ensembles are popular methods for the machine learning tasks of classification and regression. Decision trees are widely used since they are easy to interpret, handle categorical features, extend to the multiclass classification setting, do not require feature scaling, and are able to capture non-linearities and feature interactions.

 - **Random Forest Classifier**: A random forest is an ensemble of decision trees. Random forests combine many decision trees in order to reduce the risk of overfitting. Tree ensemble algorithms such as random forests and boosting are among the top performers for classification and regression tasks.

We will to compare these models in the AML Workbench _runs_ screen. The next code block creates the model. You can choose between a _DecisionTree_ or _RandomForest_ by setting the 'model_type' variable. We have also included a series of model hyperparameters to guide your exploration of the model space.

In [12]:
model_type = dbutils.widgets.get("model")

# train a model.
if model_type == 'DecisionTree':
  model = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
                                 # Maximum depth of the tree. (>= 0) 
                                 # E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'
                                 maxDepth=15,
                                 # Max number of bins for discretizing continuous features. 
                                 # Must be >=2 and >= number of categories for any categorical feature.
                                 maxBins=32, 
                                 # Minimum number of instances each child must have after split. 
                                 # If a split causes the left or right child to have fewer than 
                                 # minInstancesPerNode, the split will be discarded as invalid. Should be >= 1.
                                 minInstancesPerNode=1, 
                                 # Minimum information gain for a split to be considered at a tree node.
                                 minInfoGain=0.0, 
                                 # Criterion used for information gain calculation (case-insensitive). 
                                 # Supported options: entropy, gini')
                                 impurity="gini")

  ##=======================================================================================================================
  ## GBTClassifer is only valid for Binary Classifiers, this is a multiclass (failures 1-4) so no GBTClassifier
#elif model_type == 'GBTClassifier':
#  model = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures",
#                        maxIter=200, stepSize=0.1,
#                        maxDepth=15,
#                        maxBins=32, 
#                        minInstancesPerNode=1, 
#                        minInfoGain=0.0)
  ##=======================================================================================================================
else:
  model = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", 
                                      # Passed to DecisionTreeClassifier
                                      maxDepth=15, 
                                      maxBins=32, 
                                      minInstancesPerNode=1, 
                                      minInfoGain=0.0,
                                      impurity="gini",
                                      # Number of trees to train (>= 1)
                                      numTrees=200, 
                                      # The number of features to consider for splits at each tree node. 
                                      # Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n].
                                      featureSubsetStrategy="sqrt", 
                                      # Fraction of the training data used for learning each  
                                      # decision tree, in range (0, 1].' 
                                      subsamplingRate = 0.632)

# chain indexers and model in a Pipeline
pipeline_cls_mthd = Pipeline(stages=[labelIndexer, featureIndexer, model])

# train model.  This also runs the indexers.
model_pipeline = pipeline_cls_mthd.fit(training)

## Persist the model

We'll save the latest model for use in deploying a webservice for operationalization in the next notebook. We store this local to the Jupyter notebook kernel because the model is stored in a hierarchical format that does not translate to Azure Blob storage well.

In [14]:
# save model
model_pipeline.write().overwrite().save("dbfs:/storage/models/" + model_type + ".pqt")
print("Model saved")

# Time the notebook execution. 
# This will only make sense if you "Run All" cells
toc = time.time()
print("Full run took %.2f minutes" % ((toc - tic)/60))

Model saved
Full run took 6.11 minutes

In [15]:
%sh 
#mkdir /dbfs/FileStore/models
ls -l /dbfs/storage/models

total 0
drwxr-xr-x 1 root root 0 Nov 21 17:23 RandomForest.pqt

## Conclusion

In the next notebook `Code\4_operationalization.ipynb` Jupyter notebook we will create the functions needed to operationalize and deploy any model to get realtime predictions.